In [1]:
%matplotlib inline

import numpy as np
import tensorflow as tf
import gym
import matplotlib.pyplot as plt

In [6]:
env = gym.make('CartPole-v0')

[2017-07-11 17:10:05,024] Making new env: CartPole-v0


In [27]:
hidden_size = 10

tf.reset_default_graph()

x = tf.placeholder(dtype=tf.float32,shape=[None,4])
y = tf.placeholder(dtype=tf.float32,shape=[None,2])
r = tf.placeholder(dtype=tf.float32,shape=[None,1])
w1 = tf.Variable(tf.random_normal((4,hidden_size)))
b1 = tf.Variable(tf.zeros((1,hidden_size)))
w2 = tf.Variable(tf.random_normal((hidden_size,2)))
b2 = tf.Variable(tf.zeros((1,2)))

layer1_out = tf.nn.relu(tf.matmul(x,w1) + b1)
q_out = tf.matmul(layer1_out,w2) + b2
final_out = tf.nn.softmax(tf.nn.sigmoid(q_out))
greedy_action = tf.argmax(final_out,axis=1)

init= tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    s1 = env.reset()
    done = False
    while not done:
        # e~greedy
        # sess.run(greedy_action,{x:n_s.reshape(1,4)})
        # get current q value
        a = np.random.randint(env.action_space.n)
        s2,r,done,_ = env.step(a)
        # get maximum next q value (off-policy)
        print sess.run(greedy_action,{x:n_s.reshape(1,4)})
        # update the model by temp-diff between current q and next q
        s1 = s1

[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
